
# Customer Text Summarization — Concurrency + Batching

- **Python concurrency**: `ThreadPoolExecutor` processes multiple batches in parallel
- **Batch processing**: `llm.batch(prompts)` per chunk
- **Single, simple prompt** (no complex fallbacks)


In [1]:
!pip install OpenAI
!pip install langchain
!pip install langchain_community
!pip install Cohere
!pip install langchain-openai langchain-cohere python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1
   ━━━━━━━

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [3]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

In [4]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_openai import ChatOpenAI

In [5]:

CSV_PATH     = "https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations.csv"   # <- set your CSV path
TEXT_COL     = "customer_text"
MODEL        = "gpt-4o-mini"
TEMPERATURE  = 0
TIMEOUT      = 60

BATCH_SIZE   = 40      # rows per batch call
MAX_WORKERS  = 4       # parallel batches

OUTPUT_PATH  = CSV_PATH.replace(".csv", "_with_summary_concurrent.csv")




In [6]:

# Single summarization prompt (simple)
PROMPT_TMPL = (
    "Summarize the customer's message in ONE clear sentence focusing on intent/issue and requested action. "
    "Do not add details that are not present. Return only the summary text on a single line.\n\n"
    "CUSTOMER TEXT:\n"
    '\"\"\"{text}\"\"\"\n'
    "Summary:"
)


In [7]:

# Load
df = pd.read_csv(CSV_PATH)
if TEXT_COL not in df.columns:
    raise ValueError(f"Column '{TEXT_COL}' not found in CSV.")
texts = df[TEXT_COL].astype(str).tolist()
len(texts), df.head(3)


(3,
              call_id                 topic        tone  \
 0  CALL20250915-0001    card_block_unblock       harsh   
 1  CALL20250915-0002  loan_approval_status       harsh   
 2  CALL20250915-0003  loan_approval_status  frustrated   
 
                                        customer_text  
 0  Ms. Patel: My card ending 0530 got blocked aft...  
 1  Ms. Patel: I filed a loan application 27-Aug-2...  
 2  Mr. Singh: I filed a loan application 30-Aug-2...  )

In [8]:

# Model
llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE, timeout=TIMEOUT)

def chunk_list(seq, size):
    for i in range(0, len(seq), size):
        yield i, seq[i:i+size]




In [9]:
def summarize_batch(texts):
    prompts = [PROMPT_TMPL.format(text=t) for t in texts]
    responses = llm.batch(prompts)  # keeps order
    return [r.content.strip() for r in responses]

In [10]:

# Run batches concurrently
chunks = list(chunk_list(texts, BATCH_SIZE))
summaries_out = [None] * len(texts)

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    fut_map = {ex.submit(summarize_batch, batch): (start, batch) for (start, batch) in chunks}
    for fut in as_completed(fut_map):
        start, batch = fut_map[fut]
        result = fut.result()
        summaries_out[start:start+len(batch)] = result
        print(f"Completed rows {start}–{start+len(batch)-1}")

len(summaries_out)


Completed rows 0–2


3

In [11]:

# Save
df["summary"] = summaries_out
df.to_csv("batchprocessing.csv", index=False)
print(f"Saved: {OUTPUT_PATH}")
df[[TEXT_COL, "summary"]].head(10)


Saved: https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations_with_summary_concurrent.csv


,customer_text,summary
0,Ms. Patel: My card ending 0530 got blocked aft...,The customer requests assistance to unblock th...
1,Ms. Patel: I filed a loan application 27-Aug-2...,The customer is requesting the current approva...
2,Mr. Singh: I filed a loan application 30-Aug-2...,The customer is inquiring about the approval s...
